<a href="https://colab.research.google.com/github/JayNguyen-123/SHORTEST_PATH_IN_GRID_WITH_OBSTACLES_ELIMINATION_REINFORCEMENT_LEARNING/blob/main/Shortest_Path_in_Grid_with_Obstacles_Elimination.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Shortest Path in a Grid with Obstacles Elimination
- The Shortest Path in a Grid with Obstacles Elimination problem involves finding the shortest path from a starting cell to a target cell in a 2D grid containing obstacles, where you are allowed to eliminate up to k obstacles that lie along the path. The grid is represented by an "m x n" 2D array of 0s (empty cells) and 1s (obstacle cells).
- The goal is to find the shortest path from the starting cell (0, 0) to the target cell (m-1, n-1) by traversing through empty cells while eliminating at most k obstacles. Eliminating an obstacle means converting the obstacle cell (1) to an empty cell (0) so that the path can pass through it.


### Using BFS approach for this problem


In [2]:
from typing import List
from collections import deque

class Solution:
  def shortestPath(self, grid: List[List[int]], k: int) -> int:
    rows, cols = len(grid), len(grid[0])
    target = (rows - 1, cols -1)

    # If we have sufficient quotas to eliminate the obstacles in the worst case,
    # the the shortest distance is the Manhattan distance
    if k >= rows + cols -2:
      return rows + cols - 2
    # (row, col, remaining quota to eliminate obstacles)
    state = (0, 0, k)

    # (steps, state)
    queue = deque([(0, state)])
    seen = set([state])

    while queue:
      steps, (row, col, k) = queue.popleft()

      # We reach the target here
      if (row, col) == target:
        return steps

      # Explore the four directions in the next step
      for new_row, new_col in [(row, col + 1), (row + 1, col), (row, col - 1), (row - 1, col)]:
        # If (new_rol, new_col) is within the grid boundaries
        if (0 <= new_row < rows) and (0 <= new_col < cols): # Corrected condition new_col < cols
          new_eliminations = k - grid[new_row][new_col]
          new_state = (new_row, new_col, new_eliminations)

          # Add the next move in the queue if it qualifies
          # Also, ensure new_eliminations is not negative
          if new_eliminations >= 0 and new_state not in seen:
            seen.add(new_state)
            queue.append((steps + 1, new_state))

    return -1

grid_example = [[0,0,0],[0,1,0],[0,0,0]]
k_value = 1

solution = Solution()
print(solution.shortestPath(grid_example, k_value))

4


### Solving Shortest Path in Grid with Obstacles Elimination using Reinforcement Learning
- Build a learnable agent who with trial and error learns about the environment to solve the problem.
1. Agent: An agent is a hypothetical entity that controls the course of action. Agent has two possible options : position (0, 0) that goes right or to position (0, 1) that goes down, and both these positions have different rewards.
2. Environment: The environment is the context in which our agent operates which is a two dimensional—a grid in this case.
3. State: State represents current situation of the player. In our case, it gives current position of the player and the number of remaining violations.
4. Reward System: reward is the score we get as a result of taking a certain action. In this case : -1 points for an empty cell, +20 points for reaching the destination and -10 points if we run out of the number of violations, k.

# Q Function
- Q^{\pi }(s,a) represents the expected cumulative reward an agent can achieve by taking action a in state s, while following policy π.

Where:
 - π : the policy adopted by the agent.
 - s : the current state.
 - a : the action taken by the agent in state s.




In [5]:
import numpy as np

GRID = [[0,0,0],[1,1,0],[0,0,0],[0,1,1],[0,0,0]]
NUM_ROWS = len(GRID)
NUM_COLS = len(GRID[0])
EPSILON = 0.1
LEARNING_RATE = 0.1
DISCOUNT_RATE = 0.9
K = 2

            # Right  # Left # Bottom   # Top
ACTIONS = [(0, 1), (0, -1),  (1, 0),  (-1, 0)]

ACTION_DICT = {0:"R", 1:"L", 2:"B", 3:"T"}
INV_ACTION_DICT = {v:k for k,v in ACTION_DICT.items()}

START_STATE = (0,0)
WIN_STATE = (0,4)

class Agent:
  def __init__(self):
    self.Q = np.zeros((NUM_ROWS, NUM_COLS, K + 1, len(ACTIONS)))
    self.epsilon = EPSILON
    self.learning_rate = LEARNING_RATE
    self.discount_factor = DISCOUNT_RATE
    self.actions = ACTIONS
    self.num_rows = NUM_ROWS
    self.num_cols = NUM_COLS
    self.k = K
    self.grid = GRID

  def get_reward(self, new_i, new_j, remaining_k):
    # Reward for moving
    reward = -1

    if self.grid[new_i][new_j] == 1 and remaining_k > 0:
      new_remaining_k = remaining_k -1
    else:
      new_remaining_k = remaining_k

    if (new_i, new_j) == (NUM_ROWS - 1, NUM_COLS - 1):
      reward += 20
    if new_remaining_k < 0:
      reward -= 10
    return reward, new_remaining_k

  def start(self, iterations=50):

    for iteration in range(iterations):
      total_reward = 0
      state = (0, 0, self.k)

      while (state[0], state[1]) != (NUM_ROWS -1, NUM_COLS - 1):
        i, j, remaining_k = state
        if np.random.rand() < self.epsilon:
          action = np.random.choice(len(self.actions))
        else:
          action = np.argmax(self.Q[i, j, remaining_k])

        new_i, new_j = i + self.actions[action][0], j + self.actions[action][1]

        if new_i < 0 or new_j < 0 or new_i >= self.num_rows or new_j >= self.num_cols:
          continue

        reward, new_remaining_k = self.get_reward(new_i, new_j, remaining_k)

        # Bellman equation
        td_error = reward + self.discount_factor * np.max(self.Q[new_i, new_j, new_remaining_k]) - self.Q[i, j, remaining_k, action]

        self.Q[i, j, remaining_k, action] += self.learning_rate * td_error

        total_reward += reward
        state = (new_i, new_j, remaining_k)

      if iteration % 10 == 0 or iteration % 5:
        print(f"Epsilon {iteration+1}/{iterations} - Total Reward: {total_reward}")

    return self.build_best_action_dict()


  def build_best_action_dict(self):
    HM = {}

    for i in range(self.num_rows):
      for j in range(self.num_cols):
        max_value = float('-inf')
        max_index = float('-inf')
        for k in range(self.k+1):
          index = np.argmax(self.Q[i][j][k])
          if max_value < self.Q[i][j][k][index]:
            max_value = self.Q[i][j][k][index]
            max_index = index

          print(self.Q[i][j][k])

        HM[(i,j)] = ACTION_DICT[max_index]

    return HM

def trace_best_path(policy):
    start = (0,0)
    end = (NUM_ROWS-1, NUM_COLS-1)
    path = ''

    while start != end:
      path += policy[start]

      # Corrected line: Use policy[start] (which is a string like 'R') as the key
      # for INV_ACTION_DICT to get the numerical action index.
      next_coord_index = INV_ACTION_DICT[policy[start]]
      action = ACTIONS[next_coord_index]

      start = (start[0] + action[0], start[1] + action[1])

      print(path)

    print(path)

if __name__=='__main__':
  agent = Agent()

  policy = agent.start(1000)
  print("Optimal Policy:\n", policy)

  trace_best_path(policy)


Epsilon 1/1000 - Total Reward: 0
Epsilon 2/1000 - Total Reward: -6
Epsilon 3/1000 - Total Reward: -8
Epsilon 4/1000 - Total Reward: 4
Epsilon 5/1000 - Total Reward: -2
Epsilon 7/1000 - Total Reward: -22
Epsilon 8/1000 - Total Reward: 4
Epsilon 9/1000 - Total Reward: -8
Epsilon 10/1000 - Total Reward: 4
Epsilon 11/1000 - Total Reward: 6
Epsilon 12/1000 - Total Reward: -18
Epsilon 13/1000 - Total Reward: -14
Epsilon 14/1000 - Total Reward: -2
Epsilon 15/1000 - Total Reward: -2
Epsilon 17/1000 - Total Reward: -38
Epsilon 18/1000 - Total Reward: -58
Epsilon 19/1000 - Total Reward: -6
Epsilon 20/1000 - Total Reward: 4
Epsilon 21/1000 - Total Reward: 2
Epsilon 22/1000 - Total Reward: -6
Epsilon 23/1000 - Total Reward: -4
Epsilon 24/1000 - Total Reward: 14
Epsilon 25/1000 - Total Reward: 2
Epsilon 27/1000 - Total Reward: 12
Epsilon 28/1000 - Total Reward: -6
Epsilon 29/1000 - Total Reward: 4
Epsilon 30/1000 - Total Reward: 4
Epsilon 31/1000 - Total Reward: 6
Epsilon 32/1000 - Total Reward: 12